In [2]:
import pandas as pd
from rymscraper import rymscraper, RymUrl
network = rymscraper.RymNetwork()

In [48]:
mt = pd.read_csv("metacritic_data.csv")

In [49]:
mt['tag'] = mt["Singer"] + " - " + mt["Album_Name"]
tags = list(mt["tag"])
df = pd.DataFrame()

In [50]:
for tag in tags:
  try:
    album_infos = network.get_album_infos(name=tag)
    df = pd.concat([df, pd.DataFrame([album_infos])], axis=0)
  except IndexError:
    pass

Rate-limit detected. Restarting browser.
Rate-limit detected. Restarting browser.
Rate-limit detected. Restarting browser.
Rate-limit detected. Restarting browser.
Rate-limit detected. Restarting browser.
Rate-limit detected. Restarting browser.
Rate-limit detected. Restarting browser.
Rate-limit detected. Restarting browser.
Rate-limit detected. Restarting browser.
Rate-limit detected. Restarting browser.
Rate-limit detected. Restarting browser.
Rate-limit detected. Restarting browser.
Rate-limit detected. Restarting browser.
Rate-limit detected. Restarting browser.
Rate-limit detected. Restarting browser.
Rate-limit detected. Restarting browser.
Rate-limit detected. Restarting browser.
Rate-limit detected. Restarting browser.
Rate-limit detected. Restarting browser.
Rate-limit detected. Restarting browser.
Rate-limit detected. Restarting browser.
Rate-limit detected. Restarting browser.
Rate-limit detected. Restarting browser.
Rate-limit detected. Restarting browser.
Rate-limit detec

AttributeError: 'NoneType' object has no attribute 'find_all'

In [88]:
xdf = df.reset_index()
xdf = df[['Name', 'Artist', 'Released', 'RYM Rating', 'Genres', 'Descriptors',
       'Language', 'Track listing', 'Colorscheme', 'Languages']]
xdf.reset_index()
xdf.describe()

,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Languages
count,1488,1488,1488,1486,1488,1488,1142,1488,1488,45
unique,1479,1311,247,1474,1435,1293,10,1475,1439,21
top,Girl,King Gizzard & The Lizard Wizard,2 March 2018,3.30\n \n/ 5.00.5\n\n ...,,,English,[],"[#7a7a7a, #cccccc]","English, Spanish"
freq,2,5,21,2,5,113,1130,13,49,14


In [93]:
xdf["Language"] = [xdf.iloc[i]["Languages"] if pd.isna(xdf.iloc[i]["Language"]) else xdf.iloc[i]["Language"] for i in range(len(xdf.index))]

/var/folders/qh/04ndk5cd4w10t0bhkqvrbnh40000gn/T/ipykernel_27015/1331460494.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xdf["Language"] = [xdf.iloc[i]["Languages"] if pd.isna(xdf.iloc[i]["Language"]) else xdf.iloc[i]["Language"] for i in range(len(xdf.index))]


In [96]:
xdf = xdf[['Name', 'Artist', 'Released', 'RYM Rating', 'Genres', 'Descriptors',
       'Language', 'Track listing', 'Colorscheme']]
xdf.describe()

,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme
count,1488,1488,1488,1486,1488,1488,1187,1488,1488
unique,1479,1311,247,1474,1435,1293,31,1475,1439
top,Girl,King Gizzard & The Lizard Wizard,2 March 2018,3.30\n \n/ 5.00.5\n\n ...,,,English,[],"[#7a7a7a, #cccccc]"
freq,2,5,21,2,5,113,1130,13,49


In [188]:
import re
ratings = []
num_ratings = []
for i in xdf["RYM Rating"]:
  if isinstance(i, float):
    ratings.append("")
    num_ratings.append("")
  else:
    ratings.append(float(i.rsplit('\n')[0]))
    num_ = i.rsplit('\n')[-1]
    num = int(re.findall(r"\d*,*\d+",num_)[0].replace(",",""))
    num_ratings.append(num)

In [194]:
xdf["RYM Rating"] = ratings
xdf["Num_of_RYM_Ratings"] = num_ratings

In [195]:
xdf

,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Num_of_RYM_Ratings
0,Time Will Die and Love Will Bury It,Rolo Tomassi,2 March 2018,3.53,"Post-Hardcore, Mathcore\nPost-Rock, Blackgaze,...","female vocals, anxious, ethereal, progressi...",English,"[Towards Dawn, Aftermath, Rituals, The Hollow ...","[#b5b4b4, #9b9ba3, #6a6979, #9d9d9f, #7c7b89, ...",2302
0,Honey,Robyn,26 October 2018,3.44,"Dance-Pop, Electropop, House\nDeep House, Ambi...","female vocals, atmospheric, sensual, rhythm...",English,"[Missing U, Human Being, Because It's in the M...","[#da978f, #9a4739, #bb3623, #bba597, #7a5d3d, ...",3757
0,Safe in the Hands of Love,Yves Tumor,5 September 2018,3.61,"Neo-Psychedelia, Post-Industrial, Art Pop\nHyp...","psychedelic, eclectic, surreal, atmospheric...",English,"[Faith in Nothing Except in Salvation, Economy...","[#8d989c, #214d58, #6ea3d5, #29767c, #407c96]",9785
0,Golden Hour,Kacey Musgraves,30 March 2018,3.51,"Country Pop, Singer-Songwriter\nContemporary C...","warm, mellow, bittersweet, love, female vo...",English,"[Slow Burn, Lonely Weekend, Butterflies, Oh, W...","[#169bc6, #5daac8, #fd8670, #f3b292]",6603
0,Chris,Christine and the Queens,21 September 2018,3.36,"Synthpop\nElectropop, Alternative R&B, Synth F...","LGBT, playful, love, sexual, boastful, pa...","French, English","[French Version, Comme si on s'aimait, Damn, d...","[#a19690, #868d7d, #c1bd74, #b8c0b1, #bd9d8e]",1403
...,...,...,...,...,...,...,...,...,...,...
0,Hushed and Grim,Mastodon,29 October 2021,3.52,"Progressive Metal\nProgressive Rock, Sludge Me...","male vocals, melodic, heavy, dark, passion...",English,"[Pain With an Anchor, The Crux, Sickle and Pea...","[#b6c0c9, #616b75, #4e5b64, #7a8692, #9fa8b1, ...",2677
0,Raise the Roof,Robert Plant | Alison KraussRobert PlantAlison...,19 November 2021,3.37,"Americana\nContemporary Folk, Country Rock","female vocals, male vocals, melodic",English,"[Quattro (World Drifts In), The Price of Love,...","[#6f879f, #7da0b4, #b9c6b4, #486255]",398
0,Infinite Granite,Deafheaven,20 August 2021,3.24,"Shoegaze, Dream Pop\nPost-Rock, Dream Pop, Bla...","ethereal, male vocals, atmospheric, melanch...",English,"[Shellstar, In Blur, Great Mass of Color, Nept...","[#7a7a8c, #4d96c9, #132958, #74c3e1, #8f98b5]",4917
0,The Monster Who Hated Pennsylvania,Damien Jurado,14 May 2021,3.17,"Singer-Songwriter, Indie Folk","poetic, male vocals",English,"[Helena, Tom, Dawn Pretend, Song for Langston ...","[#cabdb3, #523d22, #6d5938, #978271, #e6c487, ...",214


In [196]:
xdf.to_csv("rym.csv")